In [1]:
import sappy as sp
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
import collections
import scanpy as sc
import pandas

import warnings
warnings.filterwarnings('ignore')

model_path = "spectrum_model"
annotated_output = "spectrum_sappy.h5ad"
fov_key = "spectrum_fov_id"
target_key = "status_plus"
retrain = True

/Users/ceglian/miniforge3/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
adata = sc.read("/Users/ceglian/GitHub/sapp2/notebooks/datasets/SPECTRUM_squidpy.h5ad")

In [4]:
status_plus = ["{}_{}".format(x,y) for x,y in zip(adata.obs["consensus_signature"],adata.obs["wgd_status"])]
adata.obs["status_plus"] = status_plus
set(adata.obs["status_plus"])

{'FBI_Non-WGD',
 'FBI_WGD',
 'HRD-Del_WGD',
 'HRD-Dup_Non-WGD',
 'HRD-Dup_WGD',
 'Undetermined_WGD'}

In [5]:
adata = adata[adata.obs["status_plus"].isin(["HRD-Dup_Non-WGD","HRD-Dup_WGD"])]

df = adata.obs
for x in set(adata.obs["tumor_supersite"]):
    dfx = df[df["tumor_supersite"] == x]
    print(x, len(set(dfx["spectrum_fov_id"])))

adata = adata[adata.obs["tumor_supersite"].isin(["Adnexa"])]
set(adata.obs["status_plus"])

Upper Quadrant 39
Peritoneum 36
Adnexa 228
Omentum 104
Bowel 102
Other 39


{'HRD-Dup_Non-WGD', 'HRD-Dup_WGD'}

In [6]:
df = adata.obs
selected_fovs = []
for sample_id in set(adata.obs["spectrum_sample_id"]):
    dfx = df[df["spectrum_sample_id"]==sample_id]
    selected_fovs += list(set(dfx["spectrum_fov_id"].tolist()))[:]
len(selected_fovs)

30

In [ ]:
for x in set(adata.obs["spectrum_fov_id"]):
    if len(df[df["spectrum_fov_id"] == x].index) > 5000:
        

In [ ]:
gae = sp.ml.GAE(adata,layers=[100,100], lr=0.01)
if not retrain:
    gae.load(model_path)